In [2]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [3]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.8.0
numpy: 1.21.5


In [4]:
# CONSTANTS:
MODEL_NAME = "SimpleGCN"

In [8]:
distances_file = '/Users/varunursekar/Projects/advDeepLearning/final_project/DCRNN-master/data/sensor_graph/distances_bay_2017.csv'

distances = pd.read_csv(distances_file,header=None)


In [9]:
distances

,0,1,2
0,400001,400001,0.0
1,400017,400017,0.0
2,400030,400030,0.0
3,400030,400045,5108.4
4,400030,400065,7401.1
...,...,...,...
8353,414694,405701,1016.7
8354,414694,407710,2290.3
8355,414694,408907,8610.4
8356,414694,408911,9119.3


In [5]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 100

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(1e-3))

In [6]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Epoch 1/100


2022-03-17 10:01:28.725430: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 10/375 [..............................] - ETA: 2s - loss: 0.5788  

2022-03-17 10:01:28.952199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 8ms/step - loss: 0.2496 - val_loss: 0.2197 - lr: 0.0010
Epoch 2/100
  1/375 [..............................] - ETA: 2s - loss: 0.1690

2022-03-17 10:01:31.977795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 8ms/step - loss: 0.2065 - val_loss: 0.2076 - lr: 0.0010
Epoch 3/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1986 - val_loss: 0.2040 - lr: 0.0010
Epoch 4/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1924 - val_loss: 0.2098 - lr: 0.0010
Epoch 5/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1872 - val_loss: 0.2080 - lr: 0.0010
Epoch 6/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1815 - val_loss: 0.1969 - lr: 0.0010
Epoch 7/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1765 - val_loss: 0.1975 - lr: 0.0010
Epoch 8/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1754 - val_loss: 0.1935 - lr: 0.0010
Epoch 9/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1738 - val_loss: 0.1943 - lr: 0.0010
Epoch 10/100
375/375 [==============================] - 3s 8ms/step - loss: 0.1707 - val_loss: 0.192

In [ ]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

In [ ]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))